In [259]:
#Data class
import random
class Sentiment: #enum class for describing the score

    NEGATIVE="NEGATIVE"
    NEUTRAL="NEUTRAL"
    POSITIVE="POSITIVE"
class Review:
    def __init__(self,text,score):
        self.text=text
        self.score=score
        self.sentiment=self.get_sentiment()

    def get_sentiment(self):
        if self.score<=2:
            return Sentiment.NEGATIVE
        elif self.score==3:
            return Sentiment.NEUTRAL
        else: #score of 4 or 5
            return Sentiment.POSITIVE
class ReviewContainer:
    def __init__(self,reviews):
        self.reviews=reviews
    
    def get_text(self):
        return [x.text for x in self.reviews]   #x is the input of the text to the training, doing list comprehension to get only the elements we need in a list.

    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]  #the input of the score to the training module.

    def evenly_distribute(self): #filters the reviews and creates a balance between number of positive and negative(same number)
        negative=list(filter(lambda x: x.sentiment==Sentiment.NEGATIVE,self.reviews))
        positive=list(filter(lambda x: x.sentiment==Sentiment.POSITIVE,self.reviews))
        positive_shrink=positive[:len(negative)]
        self.reviews=negative + positive_shrink
        random.shuffle(self.reviews)

In [260]:
#Load data
import json
file_name='./data/Books_small_10000.json'
reviews_list=[]
with open(file_name) as f: #with works like a scope where the code runs only there. (like inditation or brackets)
    for line in f:
        dict_data=json.loads(line) #loads text in json style into a json dic
        reviews_list.append(Review(dict_data['reviewText'],dict_data['overall'])) #create each element as Review class
        #append inserts data to a list, here we insert a tuple of two                 elements(the text and score).
reviews_list[1].sentiment


'NEUTRAL'

In [261]:
#Prep data
from sklearn.model_selection import train_test_split
training, test= train_test_split(reviews_list,test_size=0.33,random_state=42)
train_container=ReviewContainer(training)
test_container=ReviewContainer(test)
train_container.evenly_distribute()
test_container.evenly_distribute()
train_x = train_container.get_text() 
train_y = train_container.get_sentiment()
test_x=test_container.get_text()
test_y=test_container.get_sentiment()
train_y.count(Sentiment.POSITIVE)
train_y.count(Sentiment.NEGATIVE)

436

In [262]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()
train_x_vectors=vectorizer.fit_transform(train_x)
test_x_vectors=vectorizer.transform(test_x)
# print(test_y[3])
# print(train_x_vectors[0])

To declare a class and use it, the declation needs to be above the program ofc, otherwise we wont have the definition.(which is why Review class is above)
Basically, it makes the code more neat and clean using classes instead of indexes.

In [263]:
#Classification-> Linear SVM
from sklearn import svm
clf_svm=svm.SVC(kernel='linear')
clf_svm.fit(train_x_vectors,train_y)
test_x[0]
clf_svm.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

In [264]:
#Classification-> Decision Tree
from sklearn.tree import DecisionTreeClassifier

clf_dec=DecisionTreeClassifier()
clf_dec.fit(train_x_vectors,train_y)
clf_dec.predict(test_x_vectors[3])

array(['POSITIVE'], dtype='<U8')

In [265]:
#Classification-> Naive Bayes
from sklearn.naive_bayes import GaussianNB
clf_gnb=GaussianNB()
clf_gnb.fit(train_x_vectors.toarray(),train_y)
clf_gnb.predict(test_x_vectors[3].toarray())

array(['POSITIVE'], dtype='<U8')

In [266]:
#Evaluation
#Mean Accuracy
print(clf_svm.score(test_x_vectors,test_y))
print(clf_dec.score(test_x_vectors,test_y))
print(clf_gnb.score(test_x_vectors.toarray(),test_y))

0.7980769230769231
0.6274038461538461
0.6346153846153846


In [267]:
#F1 Scores
from sklearn.metrics import f1_score

f1_score(test_y,clf_svm.predict(test_x_vectors),average=None,labels=[Sentiment.POSITIVE,Sentiment.NEUTRAL,Sentiment.NEGATIVE])

#We got 91% for positive review, but shitty 20% for neutral and negative.
#----
#after the twiching to data that we did, with evenly contributing and getting larger chunk of data, we got a 80% for positive and 0.79% for negative, quite nice in equality terms.

array([0.8028169 , 0.        , 0.79310345])

In [278]:
#Testing the ML model
test_set=['I thouroughly enjoyed this,5 stars'
,'bad book do not buy',
'horrible waste of time',
'The best thing ive ever seen.',
'0/10 weebs everywhere']
new_test=vectorizer.transform(test_set)
print(clf_svm.predict(new_test))

['POSITIVE' 'NEGATIVE' 'NEGATIVE' 'POSITIVE' 'NEGATIVE']


In [269]:
test_y.count(Sentiment.POSITIVE)

208

In [ ]:
#Tuning our model (with Grid Search)
from sklearn.model_selection import GridSearchCV
# tuned_svm=

#vectorizer.fit_transfrom explanation:

vectorizer.get_feature_names() == (
...     ['and', 'document', 'first', 'is', 'one',
...      'second', 'the', 'third', 'this'])
True

>>> X.toarray()
array([[0, 1, 1, 1, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 2, 1, 0, 1],
       [1, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 1, 0, 1]]...)
       for every row in the matrix is a representation of the list we have
       above in get_feature_names(), and every digit in the corresponding
       place defines how many times that words show up in our corresponding
       row of matrix as the row of the string in our original tested list.
#another one:
.transform(X) = Transform dictionary features into 2D feature matrix.
.fit_transform(X) = learn feature names + .transform(X)
To answer your question:

You can .transform only after learning the features using .fit. Directly applying .transform will ignore any features not encoutered in .fit and hence, would not output any classification results.
